# Notebook 2: Popularity Baseline

Este baseline recomienda los ítems más populares globalmente para cada usuario.  
Se calculan métricas sobre el conjunto de test obtenido con leave-one-out:

- Recall@K  
- NDCG@K  
- HitRate@K  

Este baseline es extremadamente ligero y rápido, y servirá como referencia mínima para comparaciones con modelos más complejos.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from tqdm import tqdm

# Carga de datos procesados y la división train/test

In [2]:
data_path = Path("../data/processed")

train = pd.read_csv(data_path / "train.csv")
test = pd.read_csv(data_path / "test.csv")
movies = pd.read_csv(data_path / "movies.csv")

train.head(), test.head()

(   userId  movieId  rating  timestamp
 0       1     3186       4  978300019
 1       1     1270       5  978300055
 2       1     1721       4  978300055
 3       1     1022       5  978300055
 4       1     2340       3  978300103,
    userId  movieId  rating  timestamp
 0       1       48       5  978824351
 1       2     1917       3  978300174
 2       3     2081       4  978298504
 3       4     1954       5  978294282
 4       5      288       2  978246585)

# Método Popularity Baseline

La popularidad se calcula como:

\[
pop(i) = |\{ (u,i) \in Train \}|
\]

Recomendamos los ítems con mayor `pop(i)` que el usuario aún no ha visto.

In [3]:
item_popularity = (
    train.groupby("movieId")["rating"]
    .count()
    .sort_values(ascending=False)
)

item_popularity.head()

movieId
2858    3410
260     2986
1196    2985
1210    2872
480     2664
Name: rating, dtype: int64

## Construcción del recomendador
Para cada usuario:
1. Tomamos el ranking global de popularidad.
2. Excluimos los ítems que ya ha consumido en `train`.
3. Nos quedamos con los *top-K*.

In [4]:
user_train_items = (
    train.groupby("userId")["movieId"]
    .apply(set)
    .to_dict()
)

## Métricas de evaluación

Calculamos métricas basadas en **ranking**, que evalúan si el modelo es capaz de recuperar el ítem real del usuario dentro de sus **K** recomendaciones principales.

- **HitRate@K**: indica si el ítem real está (1) o no (0) dentro del top-K recomendado.  
- **Recall@K**: mide la proporción de ítems relevantes recuperados en el top-K.  
- **NDCG@K**: tiene en cuenta la posición del ítem real dentro del top-K (premia que aparezca más arriba).

Dado que con leave-one-out cada usuario tiene **solo 1 ítem en test**, estas métricas se reducen a comprobar si el modelo logra recomendar correctamente ese ítem dentro de sus primeras K sugerencias.


In [5]:
def hit_rate(ranklist, gt_item):
    return 1.0 if gt_item in ranklist else 0.0

def ndcg(ranklist, gt_item):
    if gt_item in ranklist:
        idx = ranklist.index(gt_item)
        return 1 / np.log2(idx + 2)
    return 0.0

def recall(ranklist, gt_item):
    return 1.0 if gt_item in ranklist else 0.0

# Evaluación para K = [5, 10, 20, 50]

In [6]:
Ks = [5, 10, 20, 50]

results = {k: {"hit": [], "ndcg": [], "recall": []} for k in Ks}

all_items_sorted = list(item_popularity.index)

for _, row in tqdm(test.iterrows(), total=len(test)):
    user = row["userId"]
    gt_item = row["movieId"]
    
    seen = user_train_items.get(user, set())
    candidates = [i for i in all_items_sorted if i not in seen]
    
    for k in Ks:
        top_k = candidates[:k]
        results[k]["hit"].append(hit_rate(top_k, gt_item))
        results[k]["recall"].append(recall(top_k, gt_item))
        results[k]["ndcg"].append(ndcg(top_k, gt_item))

100%|██████████| 6040/6040 [00:01<00:00, 3351.50it/s]


# Resultados finales

In [7]:
import pandas as pd

summary = []

for k in Ks:
    summary.append({
        "K": k,
        "HitRate": np.mean(results[k]["hit"]),
        "Recall": np.mean(results[k]["recall"]),
        "NDCG": np.mean(results[k]["ndcg"]),
    })

df_results = pd.DataFrame(summary)
df_results

,K,HitRate,Recall,NDCG
0,5,0.020199,0.020199,0.012680
1,10,0.036424,0.036424,0.017863
2,20,0.068377,0.068377,0.025860
3,50,0.139570,0.139570,0.039936


# Guardado de resultados
Se almacenan en `/data/processed/popularity_results.csv` para análisis posteriores.

In [8]:
df_results.to_csv(data_path / "popularity_results.csv", index=False)
print("Resultados guardados en data/processed/popularity_results.csv")

Resultados guardados en data/processed/popularity_results.csv
